# Criação de Bucket e Transferência de Arquivos para o GCP Usando Python

## Libs

In [1]:
import os
import logging
from google.cloud import storage
from typing import List, Optional

## Credenciais

In [2]:
def setup_gcs_credentials(
    credentials_path: Optional[str] = None, 
    credentials_env_var: str = "GOOGLE_APPLICATION_CREDENTIALS",
    verbose: bool = True
) -> storage.Client:
    """
    Configura as credenciais do Google Cloud Storage e inicializa o cliente.
    
    Args:
        credentials_path (str, opcional): Caminho para o arquivo de credenciais JSON
        credentials_env_var (str, opcional): Nome da variável de ambiente para credenciais
        verbose (bool, opcional): Exibir logs detalhados
    
    Returns:
        storage.Client: Cliente do Google Cloud Storage
    """
    # Configurar logging
    logging.basicConfig(
        level=logging.INFO if verbose else logging.WARNING,
        format='%(asctime)s - %(levelname)s: %(message)s'
    )
    logger = logging.getLogger(__name__)

    try:
        # Se o caminho não for fornecido, tenta usar uma localização padrão
        if credentials_path is None:
            default_paths = [
                "../key.json",
                "./key.json",
                os.path.expanduser("~/key.json")
            ]
            
            for path in default_paths:
                if os.path.exists(path):
                    credentials_path = path
                    break

        # Validar existência do arquivo de credenciais
        if not credentials_path or not os.path.exists(credentials_path):
            raise FileNotFoundError(f"Arquivo de credenciais não encontrado em {credentials_path}")

        # Configurar variável de ambiente
        os.environ[credentials_env_var] = credentials_path
        logger.info(f"Credenciais configuradas: {credentials_path}")

        # Inicializar cliente do Google Cloud Storage
        gsclient = storage.Client()
        
        return gsclient

    except Exception as e:
        logger.error(f"Erro ao configurar credenciais: {e}")
        raise

def list_gcs_buckets(
    gsclient: Optional[storage.Client] = None, 
    verbose: bool = True
) -> List[str]:
    """
    Lista os buckets disponíveis no Google Cloud Storage.
    
    Args:
        gsclient (storage.Client, opcional): Cliente do GCS. Se None, será criado.
        verbose (bool, opcional): Exibir logs detalhados
    
    Returns:
        List[str]: Lista de nomes dos buckets
    """
    # Configurar logging
    logging.basicConfig(
        level=logging.INFO if verbose else logging.WARNING,
        format='%(asctime)s - %(levelname)s: %(message)s'
    )
    logger = logging.getLogger(__name__)

    try:
        # Criar cliente se não fornecido
        if gsclient is None:
            gsclient = setup_gcs_credentials()

        # Listar buckets
        buckets = list(gsclient.list_buckets())
        bucket_names = [bucket.name for bucket in buckets]

        if verbose:
            logger.info(f"Buckets encontrados ({len(bucket_names)}):")
            for name in bucket_names:
                logger.info(f"- {name}")

        return bucket_names

    except Exception as e:
        logger.error(f"Erro ao listar buckets: {e}")
        return []


### Criação do Bucket

In [3]:
def create_gcs_bucket(
    bucket_name,
    location,
    storage_class,
    uniform_access,
    verbose,
):
    """
    Cria um bucket no Google Cloud Storage com configurações personalizáveis.
    
    Args:
        bucket_name (str): Nome do bucket a ser criado
        location (str, opcional): Localização do bucket. Padrão é 'us-east1'
        storage_class (str, opcional): Classe de armazenamento. Padrão é 'STANDARD'
        uniform_access (bool, opcional): Ativar acesso uniforme ao nível do bucket. Padrão é True
        verbose (bool, opcional): Exibir logs detalhados. Padrão é True
    
    Returns:
        google.cloud.storage.Bucket: Objeto do bucket criado
    """
    # Configurar logging
    logging.basicConfig(
        level=logging.INFO if verbose else logging.WARNING,
        format='%(asctime)s - %(levelname)s: %(message)s'
    )
    logger = logging.getLogger(__name__)

    try:
        # Verificar se o bucket já existe
        if gsclient.lookup_bucket(bucket_name):
            logger.warning(f"Bucket {bucket_name} já existe. Pulando criação.")
            return gsclient.get_bucket(bucket_name)

        # Configurações do bucket
        bucket_config = {
            "location": location,
            "storage_class": storage_class
        }

        # Criar o bucket
        bucket = gsclient.create_bucket(
            bucket_name, 
            location=bucket_config["location"]
        )

        # Definir classe de armazenamento
        bucket.storage_class = bucket_config["storage_class"]
        bucket.patch()

        # Configurar acesso uniforme (opcional)
        if uniform_access:
            bucket.iam_configuration.uniform_bucket_level_access_enabled = True
            bucket.patch()
            logger.info(f"Acesso uniforme ativado para o bucket {bucket.name}.")

        logger.info(f"Bucket {bucket.name} criado com sucesso!")
        logger.info(f"Localização: {bucket.location}")
        logger.info(f"Classe de Armazenamento: {bucket.storage_class}")

        return bucket

    except Exception as e:
        logger.error(f"Erro ao criar bucket: {e}")
        raise


### Copiando arquivos locais para bucket criado

In [4]:
def upload_folder_to_gcs(local_folder, bucket_name="", verbose=False):
#def upload_folder_to_gcs(local_folder, bucket_name, gcs_folder="", verbose=False):

    """
    Carrega recursivamente todos os arquivos de uma pasta local para um bucket do Google Cloud Storage.
    
    Args:
        local_folder (str): Caminho da pasta local a ser carregada
        bucket_name (str): Nome do bucket no Google Cloud Storage
        gcs_folder (str, opcional): Prefixo da pasta no bucket. Padrão é raiz do bucket.
        verbose (bool, opcional): Se True, imprime logs detalhados. Padrão é False.
    
    Returns:
        dict: Estatísticas de upload contendo total de arquivos, arquivos enviados e falhas
    """
    try:
        # Configurar logging
        logging.basicConfig(level=logging.INFO if verbose else logging.WARNING,
                            format='%(asctime)s - %(levelname)s: %(message)s')
        logger = logging.getLogger(__name__)

        bucket = gsclient.bucket(bucket_name)

        # Contadores para acompanhamento
        total_files = 0
        uploaded_files = 0
        failed_files = 0

        # Itera por todos os arquivos na pasta local
        for root, _, files in os.walk(local_folder):
            for file in files:
                total_files += 1
                
                # Caminho completo do arquivo local
                local_file_path = os.path.join(root, file)

                # Caminho relativo do arquivo no GCS
                relative_path = os.path.relpath(local_file_path, local_folder)
                #gcs_file_path = os.path.join(gcs_folder, relative_path).replace('\\', '/')

                try:
                    # Criar o Blob no GCS
                    #blob = bucket.blob(gcs_file_path)
                    blob = bucket.blob(relative_path)
                    
                    # Subir o arquivo
                    blob.upload_from_filename(local_file_path)
                    uploaded_files += 1
                    
                    if verbose:
                        #logger.info(f"Arquivo enviado: gs://{bucket_name}/{gcs_file_path}")
                        logger.info(f"Arquivo enviado: gs://{bucket_name}")
                
                except Exception as file_error:
                    logger.error(f"Erro ao enviar arquivo {local_file_path}: {file_error}")
                    failed_files += 1

        # Resumo do upload
        logger.info(f"Upload concluído. Total de arquivos: {total_files}, Enviados: {uploaded_files}")

        # Retornar estatísticas de upload
        return {
            'total_files': total_files,
            'uploaded_files': uploaded_files,
            'failed_files': failed_files
        }

    except Exception as e:
        logging.error(f"Erro durante o upload: {e}")
        return None

In [5]:
if __name__ == "__main__":
    try:
        # Configurar logs
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s: %(message)s'
        )
        logger = logging.getLogger(__name__)

        # 1. Configurar credenciais e criar cliente
        logger.info("🔐 Configurando credenciais do Google Cloud Storage")
        gsclient = setup_gcs_credentials(
            credentials_path="../../key.json",
            verbose=True
        )

        # 2. Listar buckets existentes
        logger.info("📋 Listando buckets existentes")
        buckets = list_gcs_buckets(gsclient, verbose=True)

        # 3. Criar novo bucket
        bucket_name = "raw_retail"
        logger.info(f"🪣 Criando bucket: {bucket_name}")
        created_bucket = create_gcs_bucket(
            bucket_name=bucket_name, 
            location="us-east1", 
            storage_class="STANDARD", 
            uniform_access=True, 
            verbose=True
        )

        # 4. Fazer upload de pasta
        local_folder = "../../data/retail_db"  
        #gcs_folder = "retail_db" 
        logger.info(f"📤 Iniciando upload da pasta: {local_folder}")
        upload_stats = upload_folder_to_gcs(
            local_folder, 
            bucket_name, 
            #gcs_folder, 
            verbose=True
        )
        # Resumo final
        logger.info("✅ Processo concluído com sucesso!")
        logger.info(f"Buckets encontrados: {len(buckets)}")
        logger.info(f"Bucket criado: {bucket_name}")
        logger.info(f"Arquivos no upload: Total={upload_stats['total_files']}, "
                    f"Enviados={upload_stats['uploaded_files']}, "
                    f"Falhas={upload_stats['failed_files']}")

    except Exception as e:
        logger.error(f"❌ Erro no processo: {e}")
        # Opcional: logging de rastreamento de erro completo
        logging.exception("Detalhes do erro:")

2024-12-15 12:37:03,550 - INFO: 🔐 Configurando credenciais do Google Cloud Storage
2024-12-15 12:37:03,551 - INFO: Credenciais configuradas: ../../key.json
2024-12-15 12:37:03,557 - INFO: 📋 Listando buckets existentes
2024-12-15 12:37:04,694 - INFO: Buckets encontrados (2):
2024-12-15 12:37:04,695 - INFO: - ingestion-raw-data-retail
2024-12-15 12:37:04,695 - INFO: - raw_retail
2024-12-15 12:37:04,695 - INFO: 🪣 Criando bucket: raw_retail
2024-12-15 12:37:04,900 - WARNING: Bucket raw_retail já existe. Pulando criação.
2024-12-15 12:37:05,514 - INFO: 📤 Iniciando upload da pasta: ../../data/retail_db
2024-12-15 12:37:07,921 - INFO: Arquivo enviado: gs://raw_retail
2024-12-15 12:37:08,158 - INFO: Arquivo enviado: gs://raw_retail
2024-12-15 12:37:08,790 - INFO: Arquivo enviado: gs://raw_retail
2024-12-15 12:37:10,246 - INFO: Arquivo enviado: gs://raw_retail
2024-12-15 12:37:10,493 - INFO: Arquivo enviado: gs://raw_retail
2024-12-15 12:37:11,966 - INFO: Arquivo enviado: gs://raw_retail
2024-1